# Monitoring Amazon Sagemaker Endpoint with Hydrosphere

This notebook shows how to:
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata
* Deploy a lambda function to shadow traffic from Sagemaker endpoint to Hydrosphere platform
* Analyze inference metrics

---

## Background

Hydrosphere supports seamless integration with Amazon Sagemaker endpoints. You can setup a shadowing channel between your Sagemaker production endopint and a Hydrosphere instance to traverse all request/response pairs for more verbose traffic analysis by Hydrosphere.

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. This ability lets you seemlessly integrate with Hydrosphere. This notebook shows you the details of setting up such a shadowing channel.

---

## Setup

To get started, make sure you have these prerequisites completed.

* Specify an AWS Region to host your model.
* An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3). See the documentation for how to fine tune the permissions needed. **Important**, this role also has to perform operations with Amazon CloudFormation service and manage Amazon Lambda functions. See [hydro-integrations](https://github.com/Hydrospheredata/hydro-serving-integrations#before-you-start) repository for more details.
* Create an S3 bucket used to store the data used to train your model, any additional model data, and the data captured from model invocations. For demonstration purposes, you are using the same bucket for these. In reality, you might want to separate them with different security policies.

In [ ]:
%%time

# Handful of configuration
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region = boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = 'sagemaker/DEMO-ModelMonitor'

data_capture_prefix = '{}/datacapture'.format(prefix)
data_training_prefix = '{}/datatraining'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
s3_training_upload_path = 's3://{}/{}'.format(bucket, data_training_prefix)
print("Capture path: {}".format(s3_capture_upload_path))
print("Training path: {}".format(s3_training_upload_path))

You can quickly verify that the execution role for this notebook has the necessary permissions to proceed. Put a simple test object into the S3 bucket you speciﬁed above. If this command fails, update the role to have `s3:PutObject` permission on the bucket and try again.

In [ ]:
# Upload some test files
boto3.Session().resource('s3').Bucket(bucket).Object("test_upload/test.txt").upload_file('test_data/upload-test-file.txt')
print("Success! You are all set to proceed.")

# Capturing real-time inference data from Amazon SageMaker endpoints and shadowing it to Hydrosphere platform
Create an endpoint to showcase the data capture capability in action.

### Upload the pre-trained model to Amazon S3
This code uploads a pre-trained XGBoost model that is ready for you to deploy. This model was trained using the XGB Churn Prediction Notebook in SageMaker. You can also use your own pre-trained model in this step. If you already have a pretrained model in Amazon S3, you can add it instead by specifying the s3_key.

In [ ]:
model_file = open("model/xgb-churn-prediction-model.tar.gz", 'rb')
s3_key = os.path.join(prefix, 'xgb-churn-prediction-model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

### Deploy the model to Amazon SageMaker
Start with deploying a pre-trained churn prediction model. Here, you create the model object with the image and model data.

In [ ]:
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.amazon.amazon_estimator import get_image_uri

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz'.format(bucket, region, prefix)
image_uri = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

model = Model(image=image_uri, model_data=model_url, role=role)

To enable data capture for monitoring the model data quality, you specify the new capture option called `DataCaptureConfig`. You can capture the request payload, the response payload or both with this configuration. The capture config applies to all variants. Go ahead with the deployment.

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'DEMO-xgb-churn-pred-model-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m5.large',
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config)

## Setup traffic shadowing

First, install `hydro-integrations` SDK, which will allow you to setup shadowing channel.

In [ ]:
!pip install hydro-integrations

### Upload training data

For monitoring purposes Hydrosphere requires an access to the training data for target model. In this example we place training data close to the `data_capture_prefix` prefix, but you can setup a different S3 URI if required. The minor constraint is to have training data placed under to format it in such a way, that will prefix endpoint name before 

Note, by default `destination_s3_uri` parameter, specified in the `data_capture_config`, represents a prefix where your requests will be stored. The minor constraint is to have training data placed under endpoint name, which you are monitoring.

In [ ]:
from hydro_integrations.aws.sagemaker import TrafficShadowing

In [ ]:
training_data_file = "test_data/training-dataset-with-header.csv"

# Note, we are including endpoint_name to training path
training_data_key = os.path.join(data_training_prefix, endpoint_name, "training-dataset-with-header.csv")

boto3.Session().resource('s3').Object(bucket, training_data_key).upload_file(training_data_file)

### Deploy traffic shadowing function

Once the endpoint has been provisioned and training data has been uploaded, we can deploy a lambda function, which will shadow traffic to Hydrosphere.

In [ ]:
hydrosphere_endpoint = "<hydrosphere>" # Update with your Hydrosphere instance endpoint

In [ ]:
shadowing = TrafficShadowing(
    hydrosphere_endpoint,
    s3_training_upload_path,
    data_capture_config,
)

shadowing.deploy()

Now open AWS Management Console and go to CloudFormation service. You will see a new stack `traffic-shadowing-hydrosphere-...` being provisioned. This stack spins up a Lambda function, which will listen to events under `s3_capture_upload_path` and notify Hydrosphere.

## Invoke the deployed model

You can now send data to this endpoint to get inferences in real time. Because you enabled the data capture in the previous steps, the request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location you have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 2 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [ ]:
from sagemaker.predictor import RealTimePredictor
import time

predictor = RealTimePredictor(endpoint=endpoint_name,content_type='text/csv')

# get a subset of test data for a quick test
!head -120 test_data/test-dataset-input-cols.csv > test_data/test_sample.csv
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open('test_data/test_sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        response = predictor.predict(data=payload)
        time.sleep(0.5)

print("Done!")        

## View Hydrosphere dashboard

Now, open Hydrosphere endpoint and navigate to the model with the same name as your Sagemaker endpoint name. Within this model collection you will see a model version with ID 1 (which is latest for this model), where you can find all details about that external model. You can open monitoring dashboard and observe captured requests.

## Delete the resources

In [ ]:
shadowing.delete()

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_model()